In [ ]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import Sequence
import ast  

# Load Model 1
model_1 = load_model('stage_1_model.h5')

# Function to generate filenames
def generate_time_file_names(start_time_str, end_time_str, folder_path, prefix='optimal_file_data_'):
    start_time = datetime.strptime(start_time_str, '%H_%M_%S')
    end_time = datetime.strptime(end_time_str, '%H_%M_%S')
    current_time = start_time
    while current_time <= end_time:
        time_str = current_time.strftime('%H_%M_%S')
        file_name = f"{prefix}{time_str}.csv"
        yield os.path.join(folder_path, file_name)
        current_time += timedelta(seconds=20)

# Function to process each file
def process_file(file_name):
    if not os.path.exists(file_name):
        return None, None, None, None

    optimal_data = pd.read_csv(file_name, usecols=['feed_sat', 'Latitude', 'Longitude', 'Altitude', 'optimal_gateway_matrix'])
    optimal_data.drop_duplicates(subset=['feed_sat'], inplace=True)

    if optimal_data.empty:
        return None, None, None, None  # Avoid processing empty files

    # Ensure all values are numeric
    optimal_data[['Latitude', 'Longitude', 'Altitude']] = optimal_data[['Latitude', 'Longitude', 'Altitude']].astype(float)

    # Extract satellite positions
    sat_positions = {row['feed_sat']: row[['Latitude', 'Longitude', 'Altitude']].values for _, row in optimal_data.iterrows()}

    # Convert labels from string to array
    sat_labels = {row['feed_sat']: np.array(ast.literal_eval(row['optimal_gateway_matrix']), dtype=np.float32) 
                  for _, row in optimal_data.iterrows()}
    
    sat_list = list(sat_positions.keys())

    if not sat_positions:
        return None, None, None, None  # Skip empty datasets

    # Convert positions to NumPy array
    X_input = np.array(list(sat_positions.values()), dtype=np.float32)

    # Ensure valid input
    if X_input.ndim != 2 or X_input.shape[1] != 3:
        print(f"Invalid shape for X_input: {X_input.shape} in {file_name}")
        return None, None, None, None  # Skip malformed data

    # Get top-3 predicted gateways
    top_3_preds = get_top_3_predictions(model_1, X_input)


    # Map predictions back to satellites
    sat_top_3_map = {sat: pred for sat, pred in zip(sat_list, top_3_preds)}

    # Find neighbors only from available satellites
    sat_neighbors = {}
    for sat, pos in sat_positions.items():
        distances = [(other_sat, np.linalg.norm(pos - sat_positions[other_sat])) 
                     for other_sat in sat_list if other_sat != sat]
        nearest_neighbors = sorted(distances, key=lambda x: x[1])[:4]  # Top-4 closest
        sat_neighbors[sat] = [sat_positions[n[0]] for n in nearest_neighbors]

    return sat_positions, sat_neighbors, sat_top_3_map, sat_labels

# Function to get top-3 predictions
def get_top_3_predictions(model, X):
    predictions = model.predict(X, batch_size=256, verbose=0)
    top_3_indices = np.argsort(predictions, axis=1)[:, -3:]
    top_3_probs = np.take_along_axis(predictions, top_3_indices, axis=1)
    return np.hstack((top_3_indices, top_3_probs))

# Training settings
folder_path = r"C:\Users\aruna\Desktop\MS Thesis\Real Data\Files with position"
train_files = list(generate_time_file_names('16_00_00', '19_00_00', folder_path))
# Initialize MinMaxScaler before using it
scaler = MinMaxScaler()  

X_train, y_train = [], []
for file_name in train_files:
    sat_positions, sat_neighbors, sat_top_3_map, sat_labels = process_file(file_name)
    if sat_positions is None:
        continue  

    for sat in sat_positions.keys():
        sat_pos = sat_positions[sat]
        neighbor_pos = sat_neighbors[sat]
        top_3_preds = sat_top_3_map[sat]
        y_label = sat_labels[sat]

        feature_vector = np.concatenate([sat_pos] + neighbor_pos + [top_3_preds])

        X_train.append(feature_vector)
        y_train.append(y_label)

X_train = np.array(X_train)
y_train = np.vstack(y_train)

# ✅ **Ensure Scaler is Fitted on Full 21 Features**
assert X_train.shape[1] == 21, "Training data should have 21 features!"
scaler.fit(X_train)  # Fit only after full training feature vector is ready
X_train = scaler.transform(X_train)


#Testing settings
# Generate test file names (similar to training)
test_files = list(generate_time_file_names('19_00_00', '20_30_00', folder_path))

X_test, y_test = [], []
for file_name in test_files:
    sat_positions, sat_neighbors, sat_top_3_map, sat_labels = process_file(file_name)
    if sat_positions is None:
        continue  

    for sat in sat_positions.keys():
        sat_pos = sat_positions[sat]
        neighbor_pos = sat_neighbors[sat]
        top_3_preds = sat_top_3_map[sat]
        y_label = sat_labels[sat]

        feature_vector = np.concatenate([sat_pos] + neighbor_pos + [top_3_preds])
        X_test.append(feature_vector)
        y_test.append(y_label)

X_test = np.array(X_test)
y_test = np.vstack(y_test)

# ✅ Ensure Test Data Also Has 21 Features Before Scaling
assert X_test.shape[1] == 21, f"Test data should have 21 features but has {X_test.shape[1]}!"
X_test = scaler.transform(X_test)
print(f"Final X_test shape before scaling: {X_test.shape}")  # Debugging
assert X_test.shape[1] == X_train.shape[1], "Feature mismatch! Check preprocessing."

# Normalize test data using the same scaler as training
X_test = scaler.transform(X_test)


# Define Model 2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import CategoricalCrossentropy

model_2 = Sequential([
    Dense(256, kernel_regularizer='l2'),
    LeakyReLU(alpha=0.05),
    BatchNormalization(),
    Dropout(0.3),

    Dense(128, kernel_regularizer='l2'),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.3),

    Dense(64, kernel_regularizer='l2'),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.3),

    Dense(1, activation='linear')  # Single output: final gateway index
])


# Compile model
optimizer = RMSprop(learning_rate=0.0005, rho=0.9)
model_2.compile(optimizer=optimizer, 
                loss=CategoricalCrossentropy(label_smoothing=0.1), 
                metrics=['accuracy'])

# Train Model 2
history = model_2.fit(X_train, y_train, epochs=50, batch_size=256, validation_data=(X_test, y_test), verbose=1)

# Evaluate Model
train_loss, train_accuracy = model_2.evaluate(X_train, y_train, verbose=0)
test_loss, test_accuracy = model_2.evaluate(X_test, y_test, verbose=0)

# Print Results
print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Save Model
model_2.save('stage_2_neigh_model.h5')
print("Model training and evaluation complete!")



In [ ]:
final_gateway_predictions = model_2.predict(X_test)  # Softmax output
final_gateways = np.argmax(final_gateway_predictions, axis=1)  # Pick highest probability

print(f"Final Predicted Gateways: {final_gateways}")


NameError: name 'X_input' is not defined

In [ ]:
# Evaluation
train_loss, train_accuracy = model_2.evaluate(X_train, y_train, verbose=0)
test_loss, test_accuracy = model_2.evaluate(X_test, y_test, verbose=0)

print(f"\nTraining Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Compute Top-K Accuracy
def compute_top_k_accuracy(model, X_test, y_test, k_values=[1, 3, 5]):
    predictions = model.predict(X_test)
    top_k_indices = {k: np.argsort(predictions, axis=1)[:, -k:] for k in k_values}
    y_true = np.argmax(y_test, axis=1)
    top_k_accuracies = {k: np.mean(np.any(top_k_indices[k] == y_true[:, None], axis=1)) * 100 for k in k_values}
    return top_k_accuracies

top_k_accuracies = compute_top_k_accuracy(model, X_test, y_test, k_values=[1, 3, 5])
print("\nTop-K Accuracy Results:")
for k, acc in top_k_accuracies.items():
    print(f"Top-{k} Accuracy: {acc:.2f}%")

import pickle

# Save scaler for Model 1
with open('stage_2_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [ ]:
import matplotlib.pyplot as plt

# Function to Plot Model Loss and Accuracy
def plot_model_history(history):
    # Plot Loss
    plt.figure(figsize=(12, 5))

    # Loss Plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Model Loss')
    plt.legend()

    # Accuracy Plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Model Accuracy')
    plt.legend()

    plt.show()

# Call this function after training the model
plot_model_history(history)
